In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00


In [11]:
! pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00


In [12]:
!pip install transformers[torch] -U

In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [29]:
import pandas as pd
df=pd.read_csv('train.csv')

In [30]:
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [31]:
df['input_col'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [32]:
df.head()

,id,anchor,target,context,score,input_col
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement


In [33]:
df.head()

,id,anchor,target,context,score,input_col
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement


In [77]:
df['input_col'][1]

'TEXT1: A47; TEXT2: act of abating; ANC1: abatement'

In [34]:
model='distilbert-base-uncased'

In [35]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model)

In [36]:
def tokz_data(x):
    return tokenizer(x['input_col'])

In [37]:
tokenized_ds = ds.map(tokz_data, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [39]:
row = tokenized_ds[0]
row['input_col']

'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement'

In [40]:
tokenized_ds = tokenized_ds.rename_columns({'score':'labels'})

In [41]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [42]:
eval_df = pd.read_csv('test.csv')

In [43]:
dds = tokenized_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_col', 'input_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['labels', 'input_col', 'input_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [44]:

eval_df['input_col'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor

#2. Converting our dataframe into a dataset object and mapping the tokenizer function to it:
eval_ds = Dataset.from_pandas(eval_df).map(tokz_data, batched=True)



Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [46]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [47]:
import evaluate

accuracy = evaluate.load("accuracy")

In [48]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [51]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [53]:
model = AutoModelForSequenceClassification.from_pretrained(model, num_labels=1).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [68]:
from huggingface_hub import notebook_login

notebook_login()

In [70]:
bs = 128
epochs = 5
from transformers import AutoModelForSequenceClassification, AutoTokenizer
args = TrainingArguments(output_dir="us_patient", learning_rate=8e-5, warmup_ratio=0.1,
                        fp16=True,
                        evaluation_strategy='epoch', per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
                        num_train_epochs=epochs, weight_decay=0.01, report_to=None,push_to_hub=True)

In [71]:
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'], tokenizer=tokenizer,
                 compute_metrics=compute_metrics)

In [72]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.026837,0.968527
2,No log,0.027317,0.968527
3,0.009900,0.024884,0.968527
4,0.009900,0.024556,0.968527
5,0.005600,0.025302,0.968527


TrainOutput(global_step=1070, training_loss=0.007572459533949878, metrics={'train_runtime': 144.6371, 'train_samples_per_second': 945.608, 'train_steps_per_second': 7.398, 'total_flos': 1094956507398156.0, 'train_loss': 0.007572459533949878, 'epoch': 5.0})

In [73]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1701276051.0d8cac67db66.3626.2:   0%|          | 0.00/6.51k [00:00<?, ?B/s]

'https://huggingface.co/Sarthak7777/us_patient/tree/main/'

In [81]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 3.56201172e-01],
       [ 5.93261719e-01],
       [ 4.98291016e-01],
       [ 2.36694336e-01],
       [ 3.11660767e-03],
       [ 5.52246094e-01],
       [ 5.11718750e-01],
       [ 2.06565857e-03],
       [ 2.49267578e-01],
       [ 1.05957031e+00],
       [ 2.02636719e-01],
       [ 2.94189453e-01],
       [ 8.11035156e-01],
       [ 8.97460938e-01],
       [ 7.71484375e-01],
       [ 4.37011719e-01],
       [ 3.73535156e-01],
       [ 1.26481056e-04],
       [ 5.46875000e-01],
       [ 4.27734375e-01],
       [ 5.27343750e-01],
       [ 2.48046875e-01],
       [ 1.10412598e-01],
       [ 2.46459961e-01],
       [ 4.90722656e-01],
       [-3.71170044e-03],
       [-4.12368774e-03],
       [-4.91333008e-03],
       [-5.26905060e-04],
       [ 6.29882812e-01],
       [ 3.41064453e-01],
       [-5.66101074e-03],
       [ 7.89550781e-01],
       [ 5.24414062e-01],
       [ 2.18383789e-01],
       [ 2.62451172e-01]])

In [82]:
preds = np.clip(preds, 0, 1)
preds

array([[3.56201172e-01],
       [5.93261719e-01],
       [4.98291016e-01],
       [2.36694336e-01],
       [3.11660767e-03],
       [5.52246094e-01],
       [5.11718750e-01],
       [2.06565857e-03],
       [2.49267578e-01],
       [1.00000000e+00],
       [2.02636719e-01],
       [2.94189453e-01],
       [8.11035156e-01],
       [8.97460938e-01],
       [7.71484375e-01],
       [4.37011719e-01],
       [3.73535156e-01],
       [1.26481056e-04],
       [5.46875000e-01],
       [4.27734375e-01],
       [5.27343750e-01],
       [2.48046875e-01],
       [1.10412598e-01],
       [2.46459961e-01],
       [4.90722656e-01],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [6.29882812e-01],
       [3.41064453e-01],
       [0.00000000e+00],
       [7.89550781e-01],
       [5.24414062e-01],
       [2.18383789e-01],
       [2.62451172e-01]])